# Interactive Data Visualization by Example

## 1. Average Bechdel Score over Time

![Original Average Bechdel Test over Time Plot](original-plots/1a.png)

<table>
    <tr>
        <td>
            ![Original Average Bechdel Test over Time Plot](original-plots/1b.png)
        </td>
        <td>
            ![Original Average Bechdel Test over Time Plot](original-plots/1c.png)
        </td>
    </tr>
</table>

## 2. Bechdel Score Distribution over Time

![](original-plots/2.png)

## 3. Film Budget by Bechdel Score

![](original-plots/3.png)

## 4. Other Metrics by Bechdel Score

<table>
    <tr>
        <td>
            ![Original Average Bechdel Test over Time Plot](original-plots/4a.png)
        </td>
        <td>
            ![Original Average Bechdel Test over Time Plot](original-plots/4b.png)
        </td>
    </tr>
</table>

<table>
    <tr>
        <td>
            ![Original Average Bechdel Test over Time Plot](original-plots/4c.png)
        </td>
        <td>
            ![Original Average Bechdel Test over Time Plot](original-plots/4d.png)
        </td>
    </tr>
</table>